# Funciones de agregación y transformación

## Importamos un dataset de Kaggle

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("rohitsahoo/sales-forecasting")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'sales-forecasting' dataset.
Path to dataset files: /kaggle/input/sales-forecasting


In [ ]:
!ls /kaggle/input/sales-forecasting

train.csv


In [ ]:
# Montar la unidad
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Ahora copiar el dataset descargado a nuestro Google Drive
import os, shutil
# Carpeta destino
dest_dir = "/content/drive/MyDrive/datasets/"

# Copiar todos los archivos del dataset al destino
for file in os.listdir(path):
    shutil.copy(os.path.join(path, file), dest_dir)

# print("Archivos copiados a:", dest_dir)
# print("Contenido:", os.listdir(dest_dir))

os.listdir(dest_dir)

['netflix-shows.zip',
 'international-hotel-booking-analytics.zip',
 'netflix_titles.csv',
 'satis_clientes.csv',
 'sakila.db',
 'booking_db_nulls.sqlite',
 'booking_db.sqlite',
 'users.csv',
 'hotels.csv',
 'reviews.csv',
 'train.csv',
 'amazon.csv',
 'Walmart_Sales.csv']

In [ ]:
import pandas as pd

In [ ]:
# Usar el método read_csv de Pandas (pd) que recibe como argumento el archivo csv y retorna un dataframe
df = pd.read_csv("/content/drive/MyDrive/datasets/train.csv")
df.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales
0,1,CA-2017-152156,08/11/2017,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600
1,2,CA-2017-152156,08/11/2017,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400
2,3,CA-2017-138688,12/06/2017,16/06/2017,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036.0,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200
3,4,US-2016-108966,11/10/2016,18/10/2016,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775
4,5,US-2016-108966,11/10/2016,18/10/2016,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680


## Drop de columnas que no nos interesan


In [ ]:
df.drop(columns=["Order ID", "Order Date", "Customer ID", "Postal Code", "Product ID"], inplace=True)
df.head()

## Filtros y Transformaciones - Parte B

### Filtros convencionales

In [ ]:
# Lista solo los registros del campo State. Probar como cambia con unique
df["State"].unique()

In [ ]:
# Filtrar los registros de un estado específico, por ejemplo Indiana, Illinois, Wisconsin
df[df["State"]=="Indiana"]

In [ ]:
# Filtrar los registros de un estado específico y mejorar con transformación
df[df["State"].str.lower()=="indiana"]

In [ ]:
# Filtrar los registros de un estado usando contains
df[df["State"].str.contains("indi", case=False)]

In [ ]:
# Filtrar los registros de una lista de estados con isin()
df[df["State"].isin(["Indiana", "California"])].sort_values("State", ascending=True)

In [ ]:
# Filtrar los registros de una lista de estados con isin() y mejorar con trasnformaciones
df[df["State"].str.lower().isin(["indiana", "california"])].sort_values("State", ascending=True)

### Filtros con filter

`filter()` no usa condiciones lógicas sobre los datos (para eso se usa query(), loc[], o comparaciones booleanas).
Solo se basa en los nombres de columnas o índices.

Es muy útil para seleccionar columnas por patrón en datasets grandes.

Se puede combinar con otros métodos, por ejemplo:

In [ ]:
# Filtrar columnas por nombre exacto
df.filter(items=['Country', 'State'])

In [ ]:
# Filtrar columnas que contienen una cadena de caracteres
df.filter(like='ountry', axis=1)

In [ ]:
# Filtrar columnas con expresión regular
df.filter(regex=r'^S|s', axis=1)

In [ ]:
# Filtrar filas (usando índices)
df.filter(like='2024', axis=0)

### Filter con query

In [ ]:
# Seleecionar las ventas mayores a 1000
df.query("Sales > 1000")

In [ ]:
# Seleecionar las ventas mayores a 1000 y menores a 1500
df.query("Sales > 1000 & Sales < 1500")

In [ ]:
# Seleecionar las ventas mayores a 1000 del estado de Illinois
df.query("Sales > 1000 & State == 'Illinois'")

In [ ]:
# Seleccionar los registros cuyo campo State contenga california (ignorar case)
df.query("State.str.contains('california', case=False, na=False)")

## Agregación

Las posibles agreaciones son:
* count() cuenta los valores no nulos
* sum() suma todos los valores numericos
* mean() calcula la media aritmética
* min(), max() retorna el valor más bajo, más alto
* std() retorna la medida de dispersión

In [ ]:
df.info()

In [ ]:
# Obtener las suma de las ventas registradas por categoría
df.groupby(["Category"], as_index=False)["Sales"].sum()

In [ ]:
# Obtener la suma de las ventas registradas por estado
df.groupby(["State"], as_index=False)["Sales"].sum()

In [ ]:
# Obtener el promedio de las ventas promedio por segmento
df.groupby(["Segment"], as_index=False)["Sales"].mean()

In [ ]:
# Calular la suma de las ventas, agrupadas por categoría y por sub-categoría
df.groupby(["Category", "Sub-Category"])["Sales"].sum()

In [ ]:
# Repetir la consulta anterior usando el método agg
df.groupby(["Category", "Sub-Category"], as_index=False).agg({"Sales": "sum"})

In [ ]:
# Repetir la consulta anterior y agregar además la cantidad de la Sub-Category
df.groupby(["Category", "Sub-Category"], as_index=False).agg({"Sales": "sum", "Sub-Category": "count"})

In [ ]:
# Agrupando por Category y Sub-Category, generar 3 columnas con:
# Suma de Sales como total_sales
# Cantidad de Sales como count_sales
# Promedio de Sales como avg_sales
# Validar el promedio
df.groupby(["Category", "Sub-Category"], as_index=False).agg(
    total_sales=("Sales", "sum"),
    count_sales=("Sales", "count"),
    avg_sales=("Sales", "mean")
)

In [ ]:
# A la comparacion de ventas por Categoria y Subcategoria con categoria total, agregar una columna extra con el indice
agg_df = (df.groupby(["Category", "Sub-Category"], as_index=False)["Sales"].sum())
agg_df["sum_sales_cat"] = agg_df.groupby(["Category"])["Sales"].transform("sum")
agg_df["Sales rate"]= agg_df["Sales"] / agg_df["sum_sales_cat"]*100
agg_df

,Category,Sub-Category,Sales,sum_sales_cat,Sales rate
0,Furniture,Bookcases,113813.1987,728658.5757,15.619551
1,Furniture,Chairs,322822.7310,728658.5757,44.303703
2,Furniture,Furnishings,89212.0180,728658.5757,12.243322
3,Furniture,Tables,202810.6280,728658.5757,27.833424
4,Office Supplies,Appliances,104618.4030,705422.3340,14.830605
5,Office Supplies,Art,26705.4100,705422.3340,3.785734
6,Office Supplies,Binders,200028.7850,705422.3340,28.355891
7,Office Supplies,Envelopes,16128.0460,705422.3340,2.286296
8,Office Supplies,Fasteners,3001.9600,705422.3340,0.425555
9,Office Supplies,Labels,12347.7260,705422.3340,1.750402


## Agregaciones y transformaciones avanzadas

In [ ]:
# Analizar que porcentaje representa cada subcategoría de su máximo
agg_df = (
    df.groupby(["Category", "Sub-Category"], as_index=False)["Sales"]
      .sum()
)

# Calculamos el máximo dentro de cada categoría
agg_df["max_sales_cat"] = agg_df.groupby("Category")["Sales"].transform("max")

# # Comparamos con ese máximo
agg_df["porcentaje_vs_max"] = agg_df["Sales"] / agg_df["max_sales_cat"] * 100
agg_df